In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
    # device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
set_session(session)
tf.global_variables_initializer().run(session=session)
tf.local_variables_initializer().run(session=session)



In [ ]:
from keras.models import load_model

model = load_model("gz_dev.model")
print(model.summary())

In [ ]:
import random
import numpy as np

from trainingHelper import TrainingHelper
from trainingGame import loadTrainingGame
from mapHelper import mapHelper
import nnHelper

from os import listdir
from os.path import isfile, join

filepath = "D:/GalconZero/Games"

TRAINING_CACHE_SUFFIX = ".npz"
TRAINING_GAME_SUFFIX = ".pickle"
BATCH_SIZE = 128 #2048

def createTrainCacheFile(trainingFile):
    trainingGame = loadTrainingGame(trainingFile)
    helper = TrainingHelper(trainingGame)    
    exampleState = trainingGame.states[0]
    mapHelper.setItems(exampleState)
    
    
    trainX = helper.getTrainX()
    trainPolicy, trainValue = helper.getTrainY()
    
    trainingCacheFile = trainingFile.replace(TRAINING_GAME_SUFFIX, TRAINING_CACHE_SUFFIX)
    
    np.savez(trainingCacheFile, trainX=trainX, trainPolicy=trainPolicy, trainValue=trainValue)

def fetchTrainXY(trainingFile):
    trainingCacheFile = trainingFile.replace(TRAINING_GAME_SUFFIX, TRAINING_CACHE_SUFFIX)
    if not isfile(trainingCacheFile):
        createTrainCacheFile(trainingFile)
        
    npzFile = np.load(trainingCacheFile)
#     print('loaded ' + trainingCacheFile)
    return npzFile['trainX'], npzFile['trainPolicy'], npzFile['trainValue']
            
# def getTrainingDataForNumGames(num=-1):
#     onlyFiles = [f for f in listdir(filepath) if isfile(join(filepath, f)) and TRAINING_GAME_SUFFIX in f]
 
#     trainX = np.empty((0, 22, 14))
#     trainY = [np.empty((0, 925)), np.empty((0,))]

#     filenames = onlyFiles if num==-1 else random.sample(onlyFiles, num)
#     for filename in filenames:
#     #     print("loading training file {}".format(filename))
#         nextTrainX, policy, value = fetchTrainXY(filepath + "/" + filename)
        
#         #twice, once for regular, once for flipped Y values
#         trainX = np.concatenate((trainX, nextTrainX))
#         trainX = np.concatenate((trainX, getXAxisReflectedTrainingData(nextTrainX)))

#         #twice, once for regular, once for flipped Y values (reflecting over x axis does not change policy or value)
#         trainY = [np.concatenate((trainY[0], policy)), np.concatenate((trainY[1], value))]
#         trainY = [np.concatenate((trainY[0], policy)), np.concatenate((trainY[1], value))]

#     print("Finished loading {} training files".format(num))
#     return trainX, trainY

def getTrainingDataForNumGames(num=-1):
    onlyFiles = [f for f in listdir(filepath) if isfile(join(filepath, f)) and TRAINING_GAME_SUFFIX in f]
 
    NUM_VALIDATION_FILES = 10
    
    valTrainX = []
    valYPolicy = []
    valYValue = []
    trainX = []
    trainPolicy = []
    trainValue = []
    
    # reserve 20 games worth of files for validation only
    validationFilenames = onlyFiles[len(onlyFiles) - NUM_VALIDATION_FILES:]
    for filename in validationFilenames:
    #     print("loading training file {}".format(filename))
        nextTrainX, policy, value = fetchTrainXY(filepath + "/" + filename)
        
        #twice, once for regular, once for flipped Y values
        valTrainX.extend(nextTrainX)
        valTrainX.extend(getXAxisReflectedTrainingData(nextTrainX))

        #twice, once for regular, once for flipped Y values (reflecting over x axis does not change policy or value)
        valYPolicy.extend(policy)
        valYPolicy.extend(policy)
        valYValue.extend(value)
        valYValue.extend(value)
    
    
    
    trainFilenames = onlyFiles[:len(onlyFiles) - NUM_VALIDATION_FILES]
    
    filenames = trainFilenames if num==-1 else random.sample(trainFilenames, num)
    for filename in filenames:
    #     print("loading training file {}".format(filename))
        nextTrainX, policy, value = fetchTrainXY(filepath + "/" + filename)
        
        #twice, once for regular, once for flipped Y values
        trainX.extend(nextTrainX)
        trainX.extend(getXAxisReflectedTrainingData(nextTrainX))

        #twice, once for regular, once for flipped Y values (reflecting over x axis does not change policy or value)
        trainPolicy.extend(policy)
        trainPolicy.extend(policy)
        trainValue.extend(value)
        trainValue.extend(value)

    valTrainX = np.array(valTrainX[::10])
    valYPolicy = np.array(valYPolicy[::10])
    valYValue = np.array(valYValue[::10])
    trainX = np.array(trainX)
    trainPolicy = np.array(trainPolicy)
    trainValue = np.array(trainValue)
    
    print("Finished loading {} training files and {} validation files".format(len(filenames), len(validationFilenames)))
    return trainX, [trainPolicy, trainValue], valTrainX, [valYPolicy, valYValue]

def trainingLoop(numGamesPerLoop, numEpochs):
    trainX, [trainYPolicy, trainYValue], valTrainX, [valYPolicy, valYValue] = getTrainingDataForNumGames(numGamesPerLoop)
    
    indices = np.arange(len(trainX))
    randomSubsetIndices = np.random.choice(indices, size=int(len(trainX)/10), replace=False)
    
    trainX = np.take(trainX, randomSubsetIndices, axis=0)
    trainYPolicy = np.take(trainYPolicy, randomSubsetIndices, axis=0)  
    trainYValue = np.take(trainYValue, randomSubsetIndices, axis=0)
    
    return model.fit(trainX, [trainYPolicy, trainYValue], validation_data=(valTrainX, [valYPolicy, valYValue]), epochs=numEpochs, shuffle=True, batch_size=BATCH_SIZE)

def getXAxisReflectedTrainingData(data):
    data = data.copy()
    for sample in data:
        for row in sample:
            row[8] *= -1
            row[11] *= -1
            row[14] *= -1
    return data


In [ ]:
historyPolicyValAcc = []
historyValueValAcc = []

historyPolicyValLoss = []
historyValueValLoss = []
historyValLoss = []

def doTrain(num_loops, numGamesPerLoop,numEpochs=1):
    for i in range(num_loops):
        history = trainingLoop(numGamesPerLoop, numEpochs)

        historyPolicyValAcc.extend(history.history['val_policy_acc'])
        historyValueValAcc.extend(history.history['val_value_acc'])
        
        historyPolicyValLoss.extend(history.history['val_policy_loss'])
        historyValueValLoss.extend(history.history['val_value_loss'])
        historyValLoss.extend(history.history['val_loss'])


In [ ]:
# import tensorflow as tf

# doTrain(num_loops=1000, numGamesPerLoop=100)

# for some dumb reason, I have to do this suddenly
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
    # for some dumb reason, this fails the first time
doTrain(num_loops=100, numGamesPerLoop=20,numEpochs=1)




In [ ]:
import matplotlib.pyplot as plt 
    
plt.figure(1, figsize=(18, 18))
   
# summarize history for accuracy  
   
plt.subplot(211)  
# plt.plot(history.history['policy_acc'])  
plt.plot(historyPolicyValAcc)  
# plt.plot(history.history['value_acc'])  
plt.plot(historyValueValAcc)  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
# plt.legend(['policy_categorical_accuracy', 'val_policy_categorical_accuracy', 'value_categorical_accuracy', 'val_value_categorical_accuracy'], loc='upper left')  
plt.legend(['Validation Policy Accuracy','Validation Value Accuracy'], loc='upper left')  
   
# summarize history for loss  
   
plt.subplot(212)  
plt.plot(historyPolicyValLoss)  
plt.plot([x * 10 for x in historyValueValLoss])  
plt.plot(historyValLoss)   
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
# plt.legend(['policy_loss', 'val_policy_loss', 'value_loss','val_value_loss','loss','val_loss',], loc='upper left')  
plt.legend(['val_policy_loss','val_value_loss','val_loss'], loc='upper left')  

plt.show()  

In [ ]:
# save model and proceed to next iteration

import re
from os import listdir
from os.path import isfile, join

# find highest iteration model 

filepath = '.'

onlyFiles = [f for f in listdir(filepath) if isfile(join(filepath, f)) and 'iter' in f]
highestIteration = max([int(re.findall(r'\d+', filename)[0]) for filename in onlyFiles], key=lambda iterNum: int(iterNum))

# newModelFileName = 'gz_dev.iter{0}.model'.format(highestIteration + 1)
newModelFileName = 'gz_dev.classic1.model'.format(highestIteration + 1)
print(newModelFileName)

model.save("gz_dev.model")
model.save(newModelFileName)

In [ ]:
from trainingGame import loadTrainingGame
from trainingHelper import TrainingHelper
from mapHelper import mapHelper

game = loadTrainingGame("testGame.pickle")
exampleState = game.states[0]
mapHelper.setItems(exampleState)
helper = TrainingHelper(game) 

trainX = helper.getTrainX()
trainY = helper.getTrainY()


In [ ]:
print(trainX[299])
print(trainY)

In [ ]:
import numpy as np
print(trainY)
a = model.predict(np.array([trainX[0]]))
a[1][0][0]

In [ ]:
from keras.models import load_model

model = load_model("gz_dev.classic1.model")